# Read pcapng file

In total, there are 27 captures:

- 9 packages for each of the 3 scenarios:
  - 0: none
  - 1: sign
  - 2: sign and encrypt
- For each scenario, there are 4 captures:
  - normal
  - sniffing
  - DOS
    - Certificate Infinity Chain Loop
    - Function Call Null dereference
    - Hping3 (syn flood)
    - Open Multiple Secure Channels
    - Translate Browse Path Call Stack Overflow
  - MITM
    - ARP spoofing
    - Port stealing

## Imports

In [37]:
from pcapng import FileScanner
import pandas as pd
import numpy as np
import re
import sys
import os
import argparse

## Declarations

In [38]:
files_path = '../../Data/network_traffic/no_filter/'
filtered_files_path = '../../Data/network_traffic/opcua_filter/'
performance_files_path = '../../Data/get_performance/'

In [39]:
pcapng_list = [
    '0-dos_certificate_inf_chain_loop.pcapng',
    '0-dos_function_call_null_deref.pcapng',
    '0-dos_hping3.pcapng',
    '0-dos_open_multiple_secure_channels.pcapng',
    '0-dos_translate_browse_path_call_stack_overflow.pcapng',
    '0-mitm_arp.pcapng',
    # '0-mitm_port.pcapng',  # Null
    '0-normal_local_server.pcapng',
    '0-sniffing.pcapng',
    '1-dos_certificate_inf_chain_loop.pcapng',
    '1-dos_function_call_null_deref.pcapng',
    '1-dos_hping3.pcapng',
    '1-dos_open_multiple_secure_channels.pcapng',
    '1-dos_translate_browse_path_call_stack_overflow.pcapng',
    # '1-mitm_arp.pcapng',  # Null
    # '1-mitm_port.pcapng',  # Null
    '1-normal_local_server.pcapng',
    '1-sniffing.pcapng',
    '2-dos_certificate_inf_chain_loop.pcapng',
    '2-dos_function_call_null_deref.pcapng',
    '2-dos_hping3.pcapng',
    '2-dos_open_multiple_secure_channels.pcapng',
    '2-dos_translate_browse_path_call_stack_overflow.pcapng',
    # '2-mitm_arp.pcapng',  # Null
    # '2-mitm_port.pcapng',  # Null
    '2-normal_local_server.pcapng',
    '2-sniffing.pcapng',
]

pcapng_filtered_list = [
    '0-dos_certificate_inf_chain_loop-filtered.pcapng',
    '0-dos_function_call_null_deref-filtered.pcapng',
    '0-dos_hping3-filtered.pcapng',
    '0-dos_open_multiple_secure_channels-filtered.pcapng',
    '0-dos_translate_browse_path_call_stack_overflow-filtered.pcapng',
    '0-mitm_arp-filtered.pcapng',
    # '0-mitm_port-filtered.pcapng',  # Null
    '0-normal_local_server-filtered.pcapng',
    '0-sniffing-filtered.pcapng',
    '1-dos_certificate_inf_chain_loop-filtered.pcapng',
    '1-dos_function_call_null_deref-filtered.pcapng',
    '1-dos_hping3-filtered.pcapng',
    '1-dos_open_multiple_secure_channels-filtered.pcapng',
    '1-dos_translate_browse_path_call_stack_overflow-filtered.pcapng',
    # '1-mitm_arp-filtered.pcapng',  # Null
    # '1-mitm_port-filtered.pcapng',  # Null
    '1-normal_local_server-filtered.pcapng',
    '1-sniffing-filtered.pcapng',
    '2-dos_certificate_inf_chain_loop-filtered.pcapng',
    '2-dos_function_call_null_deref-filtered.pcapng',
    '2-dos_hping3-filtered.pcapng',
    '2-dos_open_multiple_secure_channels-filtered.pcapng',
    '2-dos_translate_browse_path_call_stack_overflow-filtered.pcapng',
    # '2-mitm_arp-filtered.pcapng',  # Null
    # '2-mitm_port-filtered.pcapng',  # Null
    '2-normal_local_server-filtered.pcapng',
    '2-sniffing-filtered.pcapng',
]

In [40]:
csv_list = [
    '0-dos_certificate_inf_chain_loop.csv',
    '0-dos_function_call_null_deref.csv',
    '0-dos_hping3.csv',
    '0-dos_open_multiple_secure_channels.csv',
    '0-dos_translate_browse_path_call_stack_overflow.csv',
    '0-mitm_arp.csv',
    # '0-mitm_port.csv',  # Null
    '0-normal_local_server.csv',
    '0-sniffing.csv',
    '1-dos_certificate_inf_chain_loop.csv',
    '1-dos_function_call_null_deref.csv',
    '1-dos_hping3.csv',
    '1-dos_open_multiple_secure_channels.csv',
    '1-dos_translate_browse_path_call_stack_overflow.csv',
    # '1-mitm_arp.csv',  # Null
    # '1-mitm_port.csv',  # Null
    '1-normal_local_server.csv',
    '1-sniffing.csv',
    '2-dos_certificate_inf_chain_loop.csv',
    '2-dos_function_call_null_deref.csv',
    '2-dos_hping3.csv',
    '2-dos_open_multiple_secure_channels.csv',
    '2-dos_translate_browse_path_call_stack_overflow.csv',
    # '2-mitm_arp.csv',  # Null
    # '2-mitm_port.csv',  # Null
    '2-normal_local_server.csv',
    '2-sniffing.csv',
]

csv_filtered_list = [
    '0-dos_certificate_inf_chain_loop-filtered.csv',
    '0-dos_function_call_null_deref-filtered.csv',
    '0-dos_hping3-filtered.csv',
    '0-dos_open_multiple_secure_channels-filtered.csv',
    '0-dos_translate_browse_path_call_stack_overflow-filtered.csv',
    '0-mitm_arp-filtered.csv',
    # '0-mitm_port-filtered.csv',  # Null
    '0-normal_local_server-filtered.csv',
    '0-sniffing-filtered.csv',
    '1-dos_certificate_inf_chain_loop-filtered.csv',
    '1-dos_function_call_null_deref-filtered.csv',
    '1-dos_hping3-filtered.csv',
    '1-dos_open_multiple_secure_channels-filtered.csv',
    '1-dos_translate_browse_path_call_stack_overflow-filtered.csv',
    # '1-mitm_arp-filtered.csv',  # Null
    # '1-mitm_port-filtered.csv',  # Null
    '1-normal_local_server-filtered.csv',
    '1-sniffing-filtered.csv',
    '2-dos_certificate_inf_chain_loop-filtered.csv',
    '2-dos_function_call_null_deref-filtered.csv',
    '2-dos_hping3-filtered.csv',
    '2-dos_open_multiple_secure_channels-filtered.csv',
    '2-dos_translate_browse_path_call_stack_overflow-filtered.csv',
    # '2-mitm_arp-filtered.csv',  # Null
    # '2-mitm_port-filtered.csv',  # Null
    '2-normal_local_server-filtered.csv',
    '2-sniffing-filtered.csv',
]

In [41]:
performance_list = [
    '0-dos_certificate_inf_chain_loop.csv',
    '0-dos_function_call_null_deref.csv',
    '0-dos_hping3.csv',
    '0-dos_open_multiple_secure_channels.csv',
    '0-dos_translate_browse_path_call_stack_overflow.csv',
    '0-mitm_arp.csv',
    '0-mitm_port.csv',
    # '0-normal_local_server.csv',
    # '0-sniffing.csv',
    '1-dos_certificate_inf_chain_loop.csv',
    '1-dos_function_call_null_deref.csv',
    '1-dos_hping3.csv',
    '1-dos_open_multiple_secure_channels.csv',
    '1-dos_translate_browse_path_call_stack_overflow.csv',
    '1-mitm_arp.csv',
    '1-mitm_port.csv',
    # '1-normal_local_server.csv',
    # '1-sniffing.csv',
    '2-dos_certificate_inf_chain_loop.csv',
    '2-dos_function_call_null_deref.csv',
    '2-dos_hping3.csv',
    '2-dos_open_multiple_secure_channels.csv',
    '2-dos_translate_browse_path_call_stack_overflow.csv',
    '2-mitm_arp.csv',
    '2-mitm_port.csv',
    # '2-normal_local_server.csv',
    # '2-sniffing.csv',
]

## Operations

- Creating a list of all the files in the directory

In [42]:
def create_dataframe(path: str, files_list: list) -> pd.DataFrame:
    dfs = []

    for elem in files_list:
        file_parts = elem.split("-")
        file_type = int(file_parts[0])
        file_name = file_parts[1].split(".")[0]
        
        df = pd.read_csv(path+elem)
        df_new = pd.DataFrame()
        df_new['Type'] = [file_type] * len(df)
        df_new['Name'] = [file_name] * len(df)
        
        df = pd.concat([df_new, df], axis=1)
        
        dfs.append(df)

    network_traffic = pd.concat(dfs, ignore_index=True)

    return network_traffic

In [43]:
network_traffic_no_filter = create_dataframe(files_path, csv_list)
network_traffic_no_filter = network_traffic_no_filter[network_traffic_no_filter['Time'] <= 60.000000]  # Prune the data to only include 1 minute
performance_results = create_dataframe(performance_files_path, performance_list)

In [44]:
performance_results.tail()

,Type,Name,Timestamp,CPU (%),Memory (%)
8823,2,mitm_port,19.507306,6.25,3.9
8824,2,mitm_port,19.630363,10.20,3.9
8825,2,mitm_port,19.749133,4.35,3.9
8826,2,mitm_port,19.871496,8.51,3.9
8827,2,mitm_port,19.992406,10.00,3.9
